In [36]:
import datetime as dt
import pandas as pd
import alphalens as al
import numpy as np
import alphalens as al
import cvxpy
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.style as style
import plotly as py
import plotly.graph_objs as go
import cvxpy as cvx
style.use('ggplot')
plt.rcParams['figure.figsize'] = (14,8)

mod_path = '/Users/richard.wuebker/Projects/price_data_utils'
if mod_path not in sys.path:
    sys.path.append(mod_path)
    
from data_loader import DataLoader
from mp_historical_prices import HistoricalPrices

In [4]:
dl = DataLoader(prices_dir='../../price_data_utils/prices', info_dir='../../price_data_utils/info', 
                analysis_date_str='2019-08-07', cached_dir='../../price_data_utils/cached')
dl.load_data()

Prediction Date: 2019-08-09


In [5]:
data = dl.data
data.head()

,mkt_cap,beta,book_value,sector,eps,month_ago,year_ago,prev_date,analysis_date,trade_date,pred_date,volume
ticker,,,,,,,,,,,,
CSCO,229986631680,0.995519,36804000000,Technology,2.882,56.189999,44.986347,52.599998,52.340000,53.160000,52.430000,24128600.0
SITC,2533060352,0.844472,1499137000,Real Estate,1.043,13.490000,NaN,13.840000,13.780000,14.030000,14.030000,1920086.0
UAL,22748133376,1.223721,10320000000,Industrials,9.838,90.669998,85.220001,85.459999,86.480003,87.629997,86.980003,2380100.0
TROW,26297821184,1.069091,6579100000,Financial Services,7.945,108.959999,112.439903,107.260002,107.040001,110.250000,108.650002,1338400.0
ISRG,61556834304,1.269715,7265300000,Healthcare,10.129,533.159973,525.359985,503.769989,505.359985,519.409973,515.500000,491700.0


In [6]:
hp = HistoricalPrices(prices_dir='../../price_data_utils/prices', 
                      cached_dir='../../price_data_utils/cached',
                      date_str='2019-08-09')
hp.get_prices()
prices = hp.prices

# look for duplicates
p = prices.reset_index()
p.loc[p.duplicated(subset=['Date', 'ticker'], keep='first')]

,index,Date,High,Low,Open,Close,Volume,Adj Close,ticker


In [7]:
prices.head()

,Date,High,Low,Open,Close,Volume,Adj Close,ticker
0,2008-01-02,320.000000,296.904999,312.381012,320.000000,379785.0,320.000000,SITC
1,2008-01-03,334.286011,323.809998,323.809998,334.286011,144585.0,334.286011,SITC
2,2008-01-04,331.333008,322.856995,326.761993,323.761993,13020.0,323.761993,SITC
3,2008-01-07,319.286011,312.381012,312.381012,319.286011,2940.0,319.286011,SITC
4,2008-01-08,318.951996,306.761993,312.856995,318.951996,25725.0,318.951996,SITC


In [8]:
def transform_prices(df, end_date_str):
    prices_temp = df.pivot(index='Date', columns='ticker', values='Adj Close')
    end_date = dt.datetime.strptime(end_date_str, '%Y-%m-%d')
    start_date = end_date - pd.DateOffset(years=3)
    prices_temp.index = pd.to_datetime(prices_temp.index)
    prices = prices_temp.loc[start_date:end_date]
    return prices.dropna(axis=1)
p = transform_prices(prices, '2018-12-31')
returns = p.pct_change().drop(p.index[0])
print(returns.shape)
returns.head()

(754, 891)


ticker,A,AA,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACC,...,YUM,Z,ZAYO,ZBH,ZBRA,ZEN,ZG,ZION,ZNGA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,-0.026788,-0.016211,-0.034002,0.011495,0.000855,-0.027515,-0.017742,-0.055826,-0.044088,-0.015965,...,-0.011499,-0.058347,-0.002256,-0.007603,-0.045370,-0.014372,-0.058756,-0.021612,-0.037313,-0.013564
2016-01-05,-0.003441,-0.045314,-0.009533,-0.006831,-0.025059,-0.004166,0.014626,-0.002816,-0.000233,0.022861,...,-0.002493,-0.005427,0.015077,0.020823,-0.025117,-0.014582,-0.019176,-0.010857,-0.007752,0.015654
2016-01-06,0.004439,-0.071197,0.017522,-0.026455,-0.019570,0.000174,-0.015286,0.003529,-0.008388,0.005528,...,-0.007080,-0.001364,0.002599,0.004330,-0.039957,-0.014408,-0.006656,-0.029145,0.003906,0.000209
2016-01-07,-0.042475,-0.039489,-0.018918,0.011073,-0.042204,-0.002963,-0.031932,-0.039625,-0.023966,-0.017448,...,-0.034116,-0.020947,-0.046296,-0.022993,-0.045316,-0.035559,-0.036013,-0.030020,-0.035019,-0.030404
2016-01-08,-0.010513,-0.024184,-0.001978,-0.021971,0.005288,-0.027268,-0.005785,0.032471,-0.020944,-0.014595,...,-0.013607,0.027907,0.000000,-0.004217,-0.002693,-0.030725,0.037793,-0.011254,-0.028226,-0.014605


In [9]:
p.head()

ticker,A,AA,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACC,...,YUM,Z,ZAYO,ZBH,ZBRA,ZEN,ZG,ZION,ZNGA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2015-12-31,40.439266,23.386749,40.752823,149.595322,99.032730,50.860046,97.531143,90.279999,41.523396,35.627289,...,48.688587,23.480000,26.590000,101.306412,69.650002,26.440001,26.040001,25.907946,2.68,46.664272
2016-01-04,39.355980,23.007635,39.367134,151.314865,99.117409,49.460629,95.800774,85.239998,39.692707,35.058495,...,48.128719,22.110001,26.530001,100.536179,66.489998,26.059999,24.510000,25.348028,2.58,46.031315
2016-01-05,39.220570,21.965065,38.991844,150.281174,96.633583,49.254574,97.201996,85.000000,39.683453,35.859982,...,48.008751,21.990000,26.930000,102.629654,64.820000,25.680000,24.040001,25.072823,2.56,46.751911
2016-01-06,39.394669,20.401209,39.675060,146.305466,94.742485,49.263153,95.716133,85.300003,39.350605,36.058201,...,47.668831,21.959999,27.000000,103.074013,62.230000,25.309999,23.879999,24.342087,2.57,46.761662
2016-01-07,37.721378,19.595585,38.924477,147.925568,90.743942,49.117207,92.659752,81.919998,38.407528,35.429070,...,46.042538,21.500000,25.750000,100.704056,59.410000,24.410000,23.020000,23.611343,2.48,45.339909


In [10]:
universe = p.loc[:,['AAPL', 'MSFT', 'TSLA']]
universe.head()

ticker,AAPL,MSFT,TSLA
Date,,,
2015-12-31,99.032730,51.508644,240.009995
2016-01-04,99.117409,50.877312,223.410004
2016-01-05,96.633583,51.109421,223.429993
2016-01-06,94.742485,50.181000,219.039993
2016-01-07,90.743942,48.435574,215.649994


In [12]:
universe.tail()

ticker,AAPL,MSFT,TSLA
Date,,,
2018-12-24,145.642090,93.384033,295.390015
2018-12-26,155.898438,99.763077,326.089996
2018-12-27,154.886688,100.378166,316.130005
2018-12-28,154.966034,99.594429,333.869995
2018-12-31,156.463837,100.765076,332.799988


In [13]:
def log_returns(series):
    return np.log(series[-1]) - np.log(series[0])

In [15]:
alpha = universe.rolling(window=252).apply(log_returns).dropna()
alpha.head()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  """Entry point for launching an IPython kernel.


ticker,AAPL,MSFT,TSLA
Date,,,
2016-12-29,0.125435,0.152594,-0.111532
2016-12-30,0.116754,0.152770,-0.044482
2017-01-03,0.144978,0.155274,-0.029247
2017-01-04,0.163622,0.169122,0.035652
2017-01-05,0.211815,0.204524,0.050191


In [17]:
alpha_ranked = alpha.rank(axis=1)
alpha_ranked.iloc[0,:]

ticker
AAPL    2.0
MSFT    3.0
TSLA    1.0
Name: 2016-12-29 00:00:00, dtype: float64

In [33]:
from scipy.stats import zscore
alpha_zscored = alpha_ranked.apply(zscore, axis=1)
alpha_vector = alpha_zscored.iloc[-1]
alpha_zscored.head()

Date
2016-12-29    [0.0, 1.224744871391589, -1.224744871391589]
2016-12-30    [0.0, 1.224744871391589, -1.224744871391589]
2017-01-03    [0.0, 1.224744871391589, -1.224744871391589]
2017-01-04    [0.0, 1.224744871391589, -1.224744871391589]
2017-01-05    [1.224744871391589, 0.0, -1.224744871391589]
dtype: object

In [34]:
n = 10
x = y = z = np.linspace(-2, 2, n)
xv, yv, zv = np.meshgrid(x, y, z, indexing='ij')
obj_val = np.full(xv.shape, np.nan)

for i in range(n):
    for j in range(n):
        for k in range(n):
            obj_val[i,j,k] = -alpha_vector[0]*xv[i,j,k]-alpha_vector[1]*yv[i,j,k]-alpha_vector[2]*zv[i,j,k]

In [41]:
trace1 = go.Scatter3d(
    x=xv.flatten(),
    y=yv.flatten(),
    z=zv.flatten(),
    mode='markers',
    marker=dict(
        size=4,
        color=obj_val.flatten(),     # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        colorbar=dict(
                title='Objective Function'
            ),
        opacity=0.4,
        showscale=True
    ),
    hoverinfo = 'text'
)

data = [trace1]
fig = go.Figure(data=data)
py.offline.iplot(fig)
plt.tight_layout()

In [48]:
def find_optimal_holdings(alpha_vector):
    x = cvx.Variable(len(alpha_vector))
    obj = cvx.Minimize(
        -alpha_vector*x
    )
    constraints = []
    
    prob = cvx.Problem(obj, constraints)
    prob.solve(verbose=True, max_iter=500)
    optimal_weights = np.asarray(x.value).flatten()

In [49]:
optimal_weights = find_optimal_holdings(alpha_vector)
print(optimal_weights)

-----------------------------------------------------------------
           OSQP v0.5.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2018
-----------------------------------------------------------------
problem:  variables n = 3, constraints m = 0
          nnz(P) + nnz(A) = 0
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 500
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on

iter   objective    pri res    dua res    rho        time
   1  -3.9192e+06   0.00e+00   1.22e+00   1.00e-01   5.69e-05s
  25  -1.0000e+20   0.00e+00   1.22e+00   1.00e-01   1.26e-04s

status:               dual infeasible
number of iterations: 25
run ti

In [51]:
rets = universe.pct_change()[1:].fillna(0)
rets.head()

ticker,AAPL,MSFT,TSLA
Date,,,
2016-01-04,0.000855,-0.012257,-0.069164
2016-01-05,-0.025059,0.004562,0.000089
2016-01-06,-0.019570,-0.018165,-0.019648
2016-01-07,-0.042204,-0.034783,-0.015477
2016-01-08,0.005288,0.003067,-0.021563


,Sharpe Ratio
smoothed_factor,0.7
